# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/home/zysuper/workspace3/LLM-quickstart/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下明夷卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

明夷卦是《易经》中的第六卦，它的卦象是由两个阴爻夹一个阳爻构成，象征着太阳被云遮蔽的状态。这种状况在自然界中常常出现，比如太阳被云遮蔽，我们看不到太阳，但太阳依然存在。因此，明夷卦象征着你表面上看不到事物，但它们仍然存在。

在《易经》中，明夷卦的卦辞描述了这样一个场景：当太阳被云遮蔽时，我们应当保持冷静，等待时机，因为太阳总会再次升起。这反映了人们在面对困难和挫折时，要有耐心，等待时机的到来，不要轻易放弃。

此外，明夷卦还涉及到一种精神境界，即在困难和挫折面前保持冷静和理智，不受外界的干扰，保持内心的宁静。这种精神境界在《易经》中被称为“明夷”，意味着明亮而宁静。

总的来说，明夷卦教导我们要在困难和挫折面前保持冷静和理智，耐心等待时机，同时要有一种积极向上的精神境界。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的蹇卦是什么？", history=history)
print(response)

蹇卦是《易经》中的第四卦，它的卦象是由两个阳爻夹一个阴爻构成，象征着行走时困难和阻碍的象征。这种状况在人们的生活中常常出现，比如在事业、生活、人际关系等方面遇到困难和阻碍。

在《易经》中，蹇卦的卦辞描述了这样一个场景：当人们行走时，可能会遇到困难和阻碍，但只要坚持不懈，努力前行，最终会克服困难，达到目的地。这反映了人们在面对困难和挫折时，要有坚定的信念和毅力，不断努力，不要轻易放弃。

此外，蹇卦还涉及到一种精神境界，即在面对困难和挫折时，要有勇气和决心，坚持不懈，克服一切困难，达到自己的目标。这种精神境界在《易经》中被称为“蹇”，意味着艰难、不屈。

总的来说，周易中的蹇卦教导我们要在面对困难和挫折时有坚定的信念和毅力，不断努力，坚持不懈，最终克服困难，实现自己的目标。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240731_173023"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下明夷卦是什么？", base_model, qlora_model, training_tag)

问题：解释下明夷卦是什么？

原始输出：
明夷卦呈现的是坤上离下的卦象，属于坎宫的游魂卦。此卦象征光明受损，如同太阳沉入地底，寓意君子遭遇困厄，告诫人们在此时应暂停进取，静观其变，以待时机。

《象辞》阐释了该卦的意象：光明隐入地下，指示君子在此情境下管理民众应当谦逊含蓄，外表看似不显山露水，内里却保持着清明与智慧，以此包容他人，亲近民众。

《断易天机》指出，明夷卦意味着明德遭受伤害，反映了君子遭受困苦的境遇，建议万事暂且停止，需耐心等待转机。

北宋邵雍进一步解释，明夷卦象征着光明被遮蔽，诸事受阻，强调应坚持正道，忍受时运不济，等待合适的时机。

台湾学者傅佩荣则从更广阔的视角解释，强调在这样的时运下应当明哲保身，保护自己免受灾难，同时财运需暗中操作，家庭关系中可能面临分离，身体健康上需注意肝部问题，以防积郁。

传统解卦观点认为，此卦为内外卦反差明显，外部环境黑暗而内部存有光明，寓意在不利环境中应低调行事，增强内在的坚韧，适时而动，以智取胜，同时提示应具有防范意识，保持逆境中的持续努力。

总之，明夷卦揭示了在困境中自我保护与智慧生存的艺术，强调顺应时势，忍辱负重，蓄势待发，以及在逆境中坚守道德信念的重要性。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240731_173023）：
[gMASK]sop 解释下明夷卦是什么？ 明夷卦呈现的是坤上离下的卦象，属于坎宫的游魂卦。此卦象征光明受损，如同太阳沉入地底，寓意君子遭遇困厄，告诫人们在此时应暂停进取，静观其变，以待时机。

《象辞》阐释了该卦的意象：光明隐入地下，指示君子在此情境下管理民众应当谦逊含蓄，外表看似不显山露水，内里却保持着清明与智慧，以此包容他人，亲近民众。

《断易天机》指出，明夷卦意味着明德遭受伤害，反映了君子遭受困苦的境遇，建议万事暂且停止，需耐心等待转机。

北宋邵雍进一步解释，明夷卦象征着光明被遮蔽，诸事受阻，强调应坚持正道，忍受时运不济，等待合适的时机。

台湾学者傅佩荣则从更广阔的视角解释，强调在这样的时运下应当明哲保身，保护自己免受灾难，同时财运需暗中操作，家庭关系中可能面临分离，身体健康上需注意肝部问题，以防积郁。

传统解卦观点认为，此卦为内外卦反差明显，外部环境黑暗而内部存有光明，寓意在不利环境中应低调行事，增

In [12]:
base_response, ft_response = compare_chatglm_results("周易中的蹇卦是什么", base_model, qlora_model, training_tag)

问题：周易中的蹇卦是什么

原始输出：
{'name': '蹇卦，其卦象呈现上坎（水）下艮（山）的异卦相叠，象征着前进道路中的重重阻碍，正如山高水深，跋涉艰难。此卦昭示应对策略与生活哲学，强调面对困境时的明智选择与道德修养的重要性。', 'content': "\n对于行动方向，卦辞提出'利西南，不利东北'，意味着向西南行进较为有利，反之东北则多有不利。同时，卦辞中提及'利见大人，贞吉'，意味着在困境中遇见或寻求有德行与权位之人的帮助，能带来吉祥与正道。\n\n《象辞》以山水之象解释蹇卦，启示君子面对逆境应反躬自省，提升自身的德行修为，以此克服外在的艰难。而在具体解读中，此卦鼓励人们在时运不济时保持坚毅与正直，无论是个人修养、事业发展、财富积累、家庭经营还是身心健康，均需谨慎行事，依靠内在的智慧与外在的合作共渡难关。\n\n《断易天机》及邵雍、傅佩荣等历代学者的解析，均强调在蹇卦状态下，个体需忍受艰难，审慎行动，不宜轻举妄动，应重视时机的选择与个人道德的提升，通过合作与忍耐，逐渐走出困境，实现转运。\n\n综上所述，蹇卦不仅是对当前困境的描述，更是一种生活智慧的传递，引导人们在逆境中自我提升，寻求内外和谐，最终转危为安，成就事业与人生。"}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240731_173023）：
[gMASK]sop 周易中的蹇卦是什么结构 蹇卦，其卦象呈现上坎（水）下艮（山）的异卦相叠，象征着前进道路中的重重阻碍，正如山高水深，跋涉艰难。此卦昭示应对策略与生活哲学，强调面对困境时的明智选择与道德修养的重要性。

对于行动方向，卦辞提出'利西南，不利东北'，意味着向西南行进较为有利，反之东北则多有不利。同时，卦辞中提及'利见大人，贞吉'，意味着在困境中遇见或寻求有德行与权位之人的帮助，能带来吉祥与正道。

《象辞》以山水之象解释蹇卦，启示君子面对逆境应反躬自省，提升自身的德行修为，以此克服外在的艰难。而在具体解读中，此卦鼓励人们在时运不济时保持坚毅与正直，无论是个人修养、事业发展、财富积累、家庭经营还是身心健康，均需谨慎行事，依靠内在的智慧与外在的合作共渡难关。

《断易天机》及邵雍、傅佩荣等历代学者的解析，均强调在蹇卦状态下，个体需忍受艰难，审慎行动，不宜轻举妄动，应重视时机的选择

In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦，作为周易六十四卦之一，蕴含着深远的哲理，其卦象由震（雷）下坎（水）上组成，象征着事物间的互动与影响。此卦教导人们在面对变化时，应具备正确的态度与行为方式，既不能独断专行，也不能随波逐流，而是要坚守道义，根据实际情况做出明智的选择与行动。

《象辞》中描述了师卦的象辞，强调天雷通宵达旦，不断努力，才能取得成功。北宋易学家邵雍则强调，富有者更应借助时机，持续积累财富，避免短视行为。

从传统解卦的角度看，师卦强调的是 leading by example 以及 the art of mentoring, suggesting that true success is achieved through self-discipline, persistent effort, and a commitment to ethical principles. The卦面告诫人们，即使处于不利局面，亦应保持冷静，协同合作，灵活应对各种变化，无论是事业发展、商业经营还是个人修养。

整体而言，师卦提醒人们，成功并非一蹴而就，而是通过持续努力、跟随正确的做法、合作与自我 discipline的结合，才能在事物互动中实现目标。这一深刻的道理，不仅对个人成长、事业进展、团体协作乃至人生决策都具有指导意义，也揭示了成功背后的深层哲学原理，即在变化与互动中持续成长与进步。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240731_173023）：
[gMASK]sop 师卦是什么？ 师卦，作为周易六十四卦之一，展现了一幅军队出行、作战的景象，其卦象由震（雷）下坎（水）上组成，象征着军事力量与自然智慧的融合。此卦强调在军事行动中应 following the wisdom of the military, 即 智胜于强，策略决定胜负，不仅胜利于强大，而且智慧能战胜一切。

《象辞》解释，师卦象征着君子通过学习和智谋，在行事上遵循道德准则，强大于众，实现和谐。卦象启示我们，运用智慧和策略能确保军事行动的胜利，同时提醒我们在其他方面也需注意和谐与智慧的应用，以取得成功。

在具体的解读中，师卦提醒人们胜于强的智能胜过一切，强调在各种行动中，智慧和策略的重要性。此卦不仅适用于军事斗争，也适用于其他各